In [8]:
import pickle
from flask import Flask,render_template, request
import ee
import numpy as np
import logging

ee.Initialize()

model = pickle.load(open("model.pkl",'rb'))

# Import the MODIS land cover collection.
lc = ee.ImageCollection('MODIS/006/MCD12Q1')

# Import the USGS ground elevation image.
elv = ee.Image('USGS/SRTMGL1_003')

# Import the Landsat 8 collection for NDVI.
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')

# Import the MERIT Hydro: global hydrography datasets for Flow Accumulation
fd = ee.Image("MERIT/Hydro/v1_0_1")

In [9]:
def get_landslider(u_lon, u_lat):

    u_poi = ee.Geometry.Point(float(u_lon), float(u_lat))
    scale = 1000  # scale in meters

    elv_urban_point = elv.sample(u_poi, scale).first().get('elevation').getInfo()
    #print('Ground elevation at urban point:', elv_urban_point, 'm')

    lc_urban_point = lc.first().sample(u_poi, scale).first().get('LC_Type1').getInfo()
    #print('Land cover value at urban point is:', lc_urban_point)

    aspect = ee.Terrain.aspect(elv)
    aspect_urban_point = aspect.sample(u_poi, scale).first().get('aspect').getInfo()
    #print(aspect_urban_point)

    slope = ee.Terrain.slope(elv)
    slope_urban_point = slope.sample(u_poi, scale).first().get('slope').getInfo()
    #print(slope_urban_point)

    img = ee.Image(l8.filterBounds(u_poi).filterDate('2021-01-01', '2021-01-31').sort('CLOUD_COVER').first())

    def meanNDVICollection(img):
        nir = img.select('B5');
        red = img.select('B4');
        ndviImage = nir.subtract(red).divide(nir.add(red)).rename('NDVI');

    # Compute the mean of NDVI over the 'region'
        ndviValue = ndviImage.reduceRegion(**{
        'geometry': img.geometry(),
        'reducer': ee.Reducer.mean(),
        'scale': 30,
        'maxPixels': 1e9
    }).get('NDVI');  # result of reduceRegion is always a dictionary, so get the element we want

        newFeature = ee.Feature(None, {
        # Adding computed NDVI value
        'NDVI': ndviValue
    }).copyProperties(img, [
        # Picking properties from original image
        'system:time_start',
        'SUN_ELEVATION'
    ])

        return newFeature

    ndvi = meanNDVICollection(img)

    ndvi = ndvi.get('NDVI').getInfo()
    #print(ndvi)

    fa = fd.select('upa')
    fa_urban_point = fa.sample(u_poi, scale).getInfo()
    fac = fa_urban_point['features'][0]['properties']['upa']
    #print(fac)
    try:
        twi = np.log(fac*1000/np.tan(slope_urban_point))
        if np.isnan(twi):
            raise ValueError
    except ValueError:
        twi = 10
        
    try:
        spi = fac*np.tan(slope_urban_point)
    except:
        spi =10

    rainfall = 28

    fs= [[elv_urban_point,slope_urban_point,aspect_urban_point,twi,spi,ndvi,rainfall,lc_urban_point]]

    pd = model.predict_proba(fs)*100
    value = {
        "Landslide" : round(pd[0][0],2),
        "No_Landslide": round(pd[0][1],2)
    }
    return value



In [10]:
with open("coordinates_3") as f:
    data=f.read()
data = data.split("\n")

In [7]:
with open('ls_coord', 'a') as f:
    for i,d in enumerate(data[::]):
        ls = get_landslider(float(d.split(",")[0]), float(d.split(",")[1]))["Landslide"]
        #print(i, end=" ")
        if ls>=60:
            f.write("["+str(float(d.split(",")[1]))+","+str(float(d.split(",")[0]))+"],")
            f.write("\n")

KeyboardInterrupt: 

In [31]:
with open('coords', 'a') as f:
    count1= 0
    count2= 0
    for x in data[::10]:
        lon1 = float(x.split(",")[0])
        lat1 = float(x.split(",")[1])
        #print(lat1,lon1)
        #f.write("["+str(float(x.split(",")[1]))+","+str(float(x.split(",")[0]))+"],")
        for y in data:
            lon2 = float(y.split(",")[0])
            lat2 = float(y.split(",")[1])
            #print(lat2,lon2)
            if lat1 == lat2 and data.index(x)!=data.index(y):
                lon = np.linspace(lon1,lon2,10)
                i = len(lon)-3
                while i>2:
                    ls = get_landslider(lon[i], lat1)["Landslide"]
                    count1 = count1+1
                    if ls>=50 and count1>5:
                        count2 = count2+1
                        print(count2,count1)
                        f.write("["+str(float(lat1))+","+str(float(lon[i]))+"],")
                        f.write("\n")
                        count1= 0
                        i= i-1

1 16
2 25
3 6
4 9
5 9
6 11


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: RuntimeWarning: invalid value encountered in log
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: RuntimeWarning: invalid value encountered in log
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: RuntimeWarning: invalid value encountered in log
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: RuntimeWarning: invalid value encountered in log


7 8


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: RuntimeWarning: invalid value encountered in log
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: RuntimeWarning: invalid value encountered in log
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: RuntimeWarning: invalid value encountered in log


8 6


...

10 9


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: RuntimeWarning: invalid value encountered in log
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: RuntimeWarning: invalid value encountered in log


11 10
12 6
13 6
14 6
15 6
16 11
17 6
18 9
19 25
20 6
21 6
22 9
23 6
24 6
25 6
26 6


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: RuntimeWarning: invalid value encountered in log


27 9
28 6
29 6
30 6
